In [1]:

import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
import re
from scipy.sparse import hstack
import nltk
from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from nltk.corpus import stopwords
from textattack.augmentation import WordNetAugmenter , EmbeddingAugmenter , EasyDataAugmenter , CharSwapAugmenter , CheckListAugmenter
import random
import pandas as pd
import nlpaug.augmenter.word as naw
from tqdm import tqdm

In [2]:
dataset=pd.read_csv("raw/Resume/Resume.csv")

In [3]:
dataset.rename(columns={'Resume_str':'Resume','Category':'Category'},inplace=True)


In [4]:
dataset['Category'].value_counts()


Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
FINANCE                   118
ADVOCATE                  118
ACCOUNTANT                118
ENGINEERING               118
CHEF                      118
AVIATION                  117
FITNESS                   117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: count, dtype: int64

In [5]:
category = dataset['Category'].value_counts().reset_index()
category

,Category,count
0,INFORMATION-TECHNOLOGY,120
1,BUSINESS-DEVELOPMENT,120
2,FINANCE,118
3,ADVOCATE,118
4,ACCOUNTANT,118
5,ENGINEERING,118
6,CHEF,118
7,AVIATION,117
8,FITNESS,117
9,SALES,116


In [6]:
def cleanResume(resumeText):
    resumeText = resumeText.lower()
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub('[^a-zA-Z]', ' ', resumeText)
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

In [7]:
dataset['Resume_Text'] = dataset['Resume'].apply(lambda w: cleanResume(w))
dataset.head()

,ID,Resume,Resume_html,Category,Resume_Text
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr administrator marketing associate hr admin...
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR,hr specialist us hr operations summary versat...
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr director summary over years experience in ...
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr specialist summary dedicated driven and dy...
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR,hr manager skill highlights hr skills hr depa...


In [9]:
oneSetOfStopWords = set(stopwords.words('english')+['``',"''"])
totalWords =[]
Sentences = dataset['Resume_Text'].values
cleanedSentences = ""
for records in Sentences:
    cleanedSentences += records
    requiredWords = nltk.word_tokenize(records)
    for word in requiredWords:
        if word not in oneSetOfStopWords and word not in string.punctuation:
            totalWords.append(word)
    
wordfreqdist = nltk.FreqDist(totalWords)
mostcommon = wordfreqdist.most_common(50)
print(mostcommon)

[('state', 16156), ('company', 15212), ('city', 15077), ('management', 12174), ('name', 11739), ('sales', 8275), ('customer', 7927), ('business', 7866), ('skills', 7729), ('new', 6409), ('service', 6223), ('team', 6073), ('development', 5731), ('training', 5637), ('experience', 5632), ('project', 5362), ('work', 4817), ('manager', 4534), ('information', 4502), ('marketing', 4489), ('financial', 4227), ('office', 4189), ('support', 4123), ('education', 4093), ('including', 4045), ('staff', 3950), ('professional', 3857), ('data', 3797), ('system', 3697), ('systems', 3559), ('university', 3477), ('managed', 3432), ('process', 3428), ('customers', 3418), ('design', 3383), ('quality', 3367), ('client', 3272), ('clients', 3244), ('developed', 3228), ('reports', 3089), ('analysis', 3052), ('operations', 3008), ('product', 2951), ('high', 2950), ('program', 2934), ('services', 2847), ('ounting', 2843), ('microsoft', 2830), ('current', 2787), ('ounts', 2782)]


In [10]:
# Remove most common words from each sentence
def remove_common_words(sentence):
    wordfreqdist = nltk.FreqDist(totalWords)
    mostcommon = wordfreqdist.most_common(5)
    words = nltk.word_tokenize(sentence)
    words = [word for word in words if word not in mostcommon]
    return ' '.join(words)

In [27]:
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset['Category'])

# Then, split the train_data into train and validation sets
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42, stratify=train_data['Category'])
# Print the sizes of each split
print("Train size:", len(train_data))
print("Test size:", len(test_data))
print("Validation size:", len(val_data))

Train size: 1589
Test size: 497
Validation size: 398


In [28]:
# Initialize augmentation techniques
augmenter_synonym = naw.SynonymAug(aug_src='wordnet')
augmenter_antonym = naw.AntonymAug()
augmenter_swap = naw.RandomWordAug(action="swap")
augmenter_delete = naw.RandomWordAug(action="delete")

def random_augmantation(text):

    random_number = random.randint(0, 3)
    if random_number == 0:
        aug_text = augmenter_synonym.augment(text)
    if random_number == 1:
        aug_text = augmenter_antonym.augment(text)
    if random_number == 2:
        aug_text = augmenter_swap.augment(text)

    if random_number == 3:
        aug_text = augmenter_delete.augment(text)
    return aug_text

In [29]:
labels = train_data.Category.unique()
resume = []
category = []
for label in tqdm(labels):
    length = len(train_data[train_data['Category']==label])
    loop_range = 120 - length
    df = train_data[train_data['Category']==label]
    for _ in range(loop_range):
        df = df.sample(frac = 1)
        text = df.iloc[0]['Resume_Text']
        aug_text = random_augmantation(text)[0]
        resume.append(aug_text)
        category.append(label)

100%|██████████| 24/24 [00:54<00:00,  2.26s/it]


In [21]:
aug_df = pd.DataFrame({"Resume_Text":resume,"Category":category})
aug_train_data = pd.concat([train_data,aug_df])

In [22]:
aug_train_data.Category.value_counts()


Category
DESIGNER                  120
BANKING                   120
BPO                       120
AGRICULTURE               120
HEALTHCARE                120
APPAREL                   120
CONSTRUCTION              120
PUBLIC-RELATIONS          120
ARTS                      120
DIGITAL-MEDIA             120
FITNESS                   120
CHEF                      120
CONSULTANT                120
ADVOCATE                  120
ENGINEERING               120
BUSINESS-DEVELOPMENT      120
HR                        120
AVIATION                  120
INFORMATION-TECHNOLOGY    120
TEACHER                   120
FINANCE                   120
ACCOUNTANT                120
SALES                     120
AUTOMOBILE                120
Name: count, dtype: int64

In [23]:
train_data.drop(["ID","Resume","Resume_html"],axis=1,inplace=True)
val_data.drop(["ID","Resume","Resume_html"],axis=1,inplace=True)
test_data.drop(["ID","Resume","Resume_html"],axis=1,inplace=True)

In [24]:
train_data.to_csv('data/processed/train.csv',index=False)
aug_train_data.to_csv('data/processed/train_aug.csv',index=False)
test_data.to_csv('data/processed/test.csv',index=False)
val_data.to_csv('data/processed/val.csv',index=False)